In [1]:
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [128]:
#計測地点のosm_idを指定
start_osm = 266396723
mid_osm = [232919754,651358177]
end_osm = 637167088
start_mesh = 50325662142
end_mesh = 50325652441

In [129]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [130]:

sttime = "2022-04-01 12:00:00"
dbname = "kddi_trkn12_change_v26"

In [131]:
datedir = "230213"
f = open(datedir+'/'+ dbname + '.csv', 'w')

cur =connection.cursor()
sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
cur.execute(sql)
for row in cur:
    stlat = row[0]
    stlng = row[1]
sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
cur.execute(sql)
for row in cur:
    edlat = row[0]
    edlng = row[1]
cur.close()

if stlat > edlat:
    wklat = stlat
    stlat = edlat
    edlat = wklat
if stlng > edlng:
    wklng = stlng
    stlng = edlng
    edlng = wklng
            
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM kddi_12_param  "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)


        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        sql += "ORDER BY trakanmap.gridcode"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                #cur.execute(sql)
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + '\n')
        #cur.execute(sql)
        #connection.commit()
                
        cur.close()
        #print(lstItems)
        dte = dte + datetime.timedelta(minutes=15)
f.close()

33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値84.82660136062194距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値49.0距離0.0 km
50325662213
存在メッシュ：50325662213値94.84860279929075距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値84.82660136062194距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値49.0距離0.0 km
50325662232
存在メッシュ：50325662232値120.6532027198531距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値77.61002381704415距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値94.84860279929075距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値84.82660136062194距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値120.6532027198531距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値94.84860279929075距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値49.0距離0.0 km
50325662232
存在メッシュ：50325662232値120.6532027198531距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値84.82660136062194距離0.14468426940260257 km
33.7984375000:3

33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値86.7059148837466距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値47.0距離0.0 km
50325662213
50325662213 is kddi none!
50325662231
存在メッシュ：50325662231値86.7059148837466距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値47.0距離0.0 km
50325662232
存在メッシュ：50325662232値126.41182976595185距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値78.70792448513174距離0.11554041512042086 km
50325662213
50325662213 is kddi none!
50325662231
存在メッシュ：50325662231値86.7059148837466距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値126.41182976595185距離0.2893685387995886 km
50325662213
50325662213 is kddi none!
50325662142
存在メッシュ：50325662142値47.0距離0.0 km
50325662232
存在メッシュ：50325662232値126.41182976595185距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値86.7059148837466距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値96.2829911868719距離0.14468426940

33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値97.73938101312186距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値61.0距離0.0 km
50325662213
存在メッシュ：50325662213値108.01672006806248距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値97.73938101312186距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値61.0距離0.0 km
50325662232
存在メッシュ：50325662232値134.4787620248175距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値90.33894162130007距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値108.01672006806248距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値97.73938101312186距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値134.4787620248175距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値108.01672006806248距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値61.0距離0.0 km
50325662232
存在メッシュ：50325662232値134.4787620248175距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値97.73938101312186距離0.14468426940260257 km
33.798437500

33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値96.5649403181217距離0.14468426940260257 km
50325662142
50325662142 is kddi none!
50325662213
存在メッシュ：50325662213値107.35295460560596距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値96.5649403181217距離0.14468426940260257 km
50325662142
50325662142 is kddi none!
50325662232
存在メッシュ：50325662232値135.12988063474634距離0.2893685387995886 km
50325662124
50325662124 is kddi none!
50325662213
存在メッシュ：50325662213値107.35295460560596距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値96.5649403181217距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値135.12988063474634距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値107.35295460560596距離0.1851577137438622 km
50325662142
50325662142 is kddi none!
50325662232
存在メッシュ：50325662232値135.12988063474634距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値96.5649403181217距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
5032566223

33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値117.37021153437198距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値82.0距離0.0 km
50325662213
存在メッシュ：50325662213値127.26454416490486距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値117.37021153437198距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値82.0距離0.0 km
50325662232
存在メッシュ：50325662232値152.7404230673709距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値110.2455649149162距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値127.26454416490486距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値117.37021153437198距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値152.7404230673709距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値127.26454416490486距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値82.0距離0.0 km
50325662232
存在メッシュ：50325662232値152.7404230673709距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値117.37021153437198距離0.14468426940260257 km
33.79843

...

33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値104.53754218124824距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値84.0距離0.0 km
50325662213
50325662213 is kddi none!
50325662231
存在メッシュ：50325662231値104.53754218124824距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値84.0距離0.0 km
50325662232
存在メッシュ：50325662232値125.07508436169923距離0.2893685387995886 km
50325662124
50325662124 is kddi none!
50325662213
50325662213 is kddi none!
50325662231
存在メッシュ：50325662231値104.53754218124824距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値125.07508436169923距離0.2893685387995886 km
50325662213
50325662213 is kddi none!
50325662142
存在メッシュ：50325662142値84.0距離0.0 km
50325662232
存在メッシュ：50325662232値125.07508436169923距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値104.53754218124824距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値105.75255905374753距離0.14468426940260257 km
50325662142
存在メ

33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値87.2089488799975距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値58.0距離0.0 km
50325662213
存在メッシュ：50325662213値95.37975260069564距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値87.2089488799975距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値58.0距離0.0 km
50325662232
存在メッシュ：50325662232値116.41789775886113距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値81.32536973618886距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値95.37975260069564距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値87.2089488799975距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値116.41789775886113距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値95.37975260069564距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値58.0距離0.0 km
50325662232
存在メッシュ：50325662232値116.41789775886113距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値87.2089488799975距離0.14468426940260257 km
33.7984375000:33

33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値108.22923696874707距離0.14468426940260257 km
50325662142
50325662142 is kddi none!
50325662213
50325662213 is kddi none!
50325662231
存在メッシュ：50325662231値108.22923696874707距離0.14468426940260257 km
50325662142
50325662142 is kddi none!
50325662232
存在メッシュ：50325662232値142.45847393616538距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値101.33441765959633距離0.11554041512042086 km
50325662213
50325662213 is kddi none!
50325662231
存在メッシュ：50325662231値108.22923696874707距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値142.45847393616538距離0.2893685387995886 km
50325662213
50325662213 is kddi none!
50325662142
50325662142 is kddi none!
50325662232
存在メッシュ：50325662232値142.45847393616538距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値108.22923696874707距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値125.10855049187174距離0.14468426940260257 

33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値115.30327927562146距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値74.0距離0.0 km
50325662213
50325662213 is kddi none!
50325662231
存在メッシュ：50325662231値115.30327927562146距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値74.0距離0.0 km
50325662232
存在メッシュ：50325662232値156.60655854963957距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値106.98353064257958距離0.11554041512042086 km
50325662213
50325662213 is kddi none!
50325662231
存在メッシュ：50325662231値115.30327927562146距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値156.60655854963957距離0.2893685387995886 km
50325662213
50325662213 is kddi none!
50325662142
存在メッシュ：50325662142値74.0距離0.0 km
50325662232
存在メッシュ：50325662232値156.60655854963957距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値115.30327927562146距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値99.59840644749696距離0.1

33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値117.24241500937275距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値91.0距離0.0 km
50325662213
存在メッシュ：50325662213値124.58337147718748距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値117.24241500937275距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値91.0距離0.0 km
50325662232
存在メッシュ：50325662232値143.4848300177268距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値111.95638687235719距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値124.58337147718748距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値117.24241500937275距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値143.4848300177268距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値124.58337147718748距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値91.0距離0.0 km
50325662232
存在メッシュ：50325662232値143.4848300177268距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値117.24241500937275距離0.14468426940260257 km
33.7984

In [132]:
connection.close()

In [23]:
#CreateGeojsonFile